In [49]:
import pandas as pd
import numpy as np
import random
from tensorflow.keras import models, Sequential, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

ImportError: ignored

In [2]:
df = pd.read_csv('Emotion_final.csv')

In [3]:
df = df.rename(columns={'Text': 'text', 'Emotion': 'emotion'})

In [4]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
df.emotion.value_counts().count()

6

In [6]:
df.shape

(21459, 2)

In [7]:
X = df.text
df.emotion = pd.Categorical(df.emotion)

In [8]:
df['code'] = df['emotion'].cat.codes

In [9]:
y = to_categorical(df['code'].values)

In [10]:
df.head()

,text,emotion,code
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
y_train

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [13]:
df.head()

,text,emotion,code
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0


In [14]:
df.emotion.value_counts()

happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: emotion, dtype: int64

In [15]:
df.head()

,text,emotion,code
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0


In [35]:
### Let's tokenize the vocabulary 
tk = Tokenizer()
tk.fit_on_texts(X)
vocab_size = len(tk.word_index) + 1
print(f'There are {vocab_size} different words in your corpus')
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)


### Pad your inputs
X_train_pad = pad_sequences(X_train_token, maxlen=66, dtype='float32', padding='post' )
X_test_pad = pad_sequences(X_test_token, maxlen=66, dtype='float32', padding='post')

There are 19259 different words in your corpus


In [36]:
X_train_pad.shape

(15021, 66)

In [37]:
X_test_pad.shape

(6438, 66)

In [18]:
vocab_size

19259

In [19]:
# model.add(layers.Masking(mark_value=-999))

In [38]:
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

In [39]:
# Vectorize
vector_size = 100
gensim_weight_matrix = np.zeros((vocab_size ,vector_size))
gensim_weight_matrix.shape
for word, index in tk.word_index.items():
    if index < vocab_size: # since index starts with zero 
        if word in glove_gensim.wv.vocab:
            gensim_weight_matrix[index] = glove_gensim[word]
        else:
            gensim_weight_matrix[index] = np.zeros(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [40]:
def create_model(EMBEDDING_DIM = 100): # this means the embedding layer will create  a vector in 100 dimension
  model_cnn = Sequential()
  model_cnn.add(layers.Embedding(input_dim = vocab_size,# the whole vocabulary size 
                            output_dim = EMBEDDING_DIM, # vector space dimension
                            input_length= X_train_pad.shape[1], # max_len of text sequence
                            weights = [gensim_weight_matrix],
                            trainable = False,
                            mask_zero=True))
  model_cnn.add(layers.Conv1D(10, kernel_size=3))
  model_cnn.add(layers.Flatten())
  #model_cnn.add(layers.Dense(60, activation="relu"))
  model_cnn.add(layers.Dense(30, activation="relu"))
  model_cnn.add(layers.Dense(6, activation="softmax"))
  model_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model_cnn

In [41]:
model = create_model()

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 66, 100)           1925900   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 64, 10)            3010      
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                19230     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 186       
Total params: 1,948,326
Trainable params: 22,426
Non-trainable params: 1,925,900
_________________________________________________________________


In [43]:
es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [26]:
history = model.fit(X_train_pad, y_train, batch_size=8, epochs=100, verbose=1, validation_split=0.3, callbacks=[es])

Epoch 1/100
1315/1315 [==============================] - 8s 4ms/step - loss: 1.4737 - accuracy: 0.4245 - val_loss: 1.3423 - val_accuracy: 0.4890
Epoch 2/100
1315/1315 [==============================] - 5s 4ms/step - loss: 1.2823 - accuracy: 0.5190 - val_loss: 1.2658 - val_accuracy: 0.5281
Epoch 3/100
1315/1315 [==============================] - 5s 4ms/step - loss: 1.1481 - accuracy: 0.5690 - val_loss: 1.2506 - val_accuracy: 0.5323
Epoch 4/100
1315/1315 [==============================] - 5s 4ms/step - loss: 1.0250 - accuracy: 0.6178 - val_loss: 1.2456 - val_accuracy: 0.5529
Epoch 5/100
1315/1315 [==============================] - 5s 4ms/step - loss: 0.9253 - accuracy: 0.6564 - val_loss: 1.2727 - val_accuracy: 0.5471
Epoch 6/100
1315/1315 [==============================] - 5s 4ms/step - loss: 0.8427 - accuracy: 0.6897 - val_loss: 1.2956 - val_accuracy: 0.5549
Epoch 7/100
1315/1315 [==============================] - 5s 4ms/step - loss: 0.7699 - accuracy: 0.7171 - val_loss: 1.3557 - val_ac

In [44]:
model.evaluate(X_test_pad, y_test)

202/202 [==============================] - 1s 3ms/step - loss: 2.0320 - accuracy: 0.0980


[2.031979560852051, 0.0980118066072464]

In [65]:
def create_model2(EMBEDDING_DIM = 100): # this means the embedding layer will create  a vector in 100 dimension
  model_cnn = Sequential()
  model_cnn.add(layers.Embedding(input_dim = vocab_size,# the whole vocabulary size 
                            output_dim = EMBEDDING_DIM, # vector space dimension
                            input_length= X_train_pad.shape[1], # max_len of text sequence
                            weights = [gensim_weight_matrix],
                            trainable = False,
                            mask_zero=True))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=True)))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=True)))
  model_cnn.add(layers.Dropout(.2,))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=False)))
  #model_cnn.add(layers.Dense(60, activation="relu"))
  model_cnn.add(layers.Dense(30, activation="relu"))
  model_cnn.add(layers.Dense(6, activation="softmax"))
  model_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model_cnn

In [66]:
model2 = create_model2()

In [67]:
es = callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [68]:
history2 = model2.fit(X_train_pad, y_train, batch_size=8, epochs=10, verbose=1, validation_split=0.3, callbacks=[es])

Epoch 1/10
1315/1315 [==============================] - 75s 43ms/step - loss: 1.3256 - accuracy: 0.4838 - val_loss: 0.9421 - val_accuracy: 0.6434
Epoch 2/10
1315/1315 [==============================] - 51s 39ms/step - loss: 0.7257 - accuracy: 0.7437 - val_loss: 0.5317 - val_accuracy: 0.8187
Epoch 3/10
1315/1315 [==============================] - 51s 38ms/step - loss: 0.3919 - accuracy: 0.8652 - val_loss: 0.3648 - val_accuracy: 0.8642
Epoch 4/10
1315/1315 [==============================] - 51s 38ms/step - loss: 0.2642 - accuracy: 0.8998 - val_loss: 0.3007 - val_accuracy: 0.8919
Epoch 5/10
1315/1315 [==============================] - 51s 39ms/step - loss: 0.1996 - accuracy: 0.9195 - val_loss: 0.2809 - val_accuracy: 0.8908
Epoch 6/10
1315/1315 [==============================] - 51s 39ms/step - loss: 0.1638 - accuracy: 0.9289 - val_loss: 0.2612 - val_accuracy: 0.9039
Epoch 7/10
1315/1315 [==============================] - 51s 39ms/step - loss: 0.1435 - accuracy: 0.9351 - val_loss: 0.2680 -

In [77]:
model2.evaluate(X_test_pad, y_test)

202/202 [==============================] - 3s 15ms/step - loss: 0.2212 - accuracy: 0.9175


[0.22117826342582703, 0.9175209403038025]

In [72]:
def create_model3(EMBEDDING_DIM = 100): # this means the embedding layer will create  a vector in 100 dimension
  model_cnn = Sequential()
  model_cnn.add(layers.Embedding(input_dim = vocab_size,# the whole vocabulary size 
                            output_dim = EMBEDDING_DIM, # vector space dimension
                            input_length= X_train_pad.shape[1], # max_len of text sequence
                            weights = [gensim_weight_matrix],
                            trainable = False,
                            mask_zero=True))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=True)))
  model_cnn.add(layers.Dropout(.3,))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=True)))
  model_cnn.add(layers.Dropout(.3,))
  model_cnn.add(layers.Bidirectional(layers.LSTM(100,return_sequences=False)))
  #model_cnn.add(layers.Dense(60, activation="relu"))
  model_cnn.add(layers.Dense(30, activation="relu"))
  model_cnn.add(layers.Dropout(.3,))
  model_cnn.add(layers.Dense(6, activation="softmax"))
  model_cnn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model_cnn

In [73]:
model3 = create_model3()

In [74]:
es = callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [75]:
history3 = model3.fit(X_train_pad, y_train, batch_size=8, epochs=10, verbose=1, validation_split=0.3, callbacks=[es])

Epoch 1/10
1315/1315 [==============================] - 75s 43ms/step - loss: 1.4290 - accuracy: 0.4398 - val_loss: 1.0356 - val_accuracy: 0.6155
Epoch 2/10
1315/1315 [==============================] - 52s 40ms/step - loss: 0.9047 - accuracy: 0.6740 - val_loss: 0.6485 - val_accuracy: 0.7826
Epoch 3/10
1315/1315 [==============================] - 53s 40ms/step - loss: 0.5492 - accuracy: 0.8162 - val_loss: 0.4939 - val_accuracy: 0.8276
Epoch 4/10
1315/1315 [==============================] - 52s 39ms/step - loss: 0.3696 - accuracy: 0.8762 - val_loss: 0.3853 - val_accuracy: 0.8724
Epoch 5/10
1315/1315 [==============================] - 52s 39ms/step - loss: 0.2892 - accuracy: 0.8981 - val_loss: 0.3554 - val_accuracy: 0.8587
Epoch 6/10
1315/1315 [==============================] - 53s 40ms/step - loss: 0.2315 - accuracy: 0.9154 - val_loss: 0.3388 - val_accuracy: 0.8893
Epoch 7/10
1315/1315 [==============================] - 52s 39ms/step - loss: 0.2003 - accuracy: 0.9244 - val_loss: 0.2547 -

In [76]:
model3.evaluate(X_test_pad, y_test)

202/202 [==============================] - 3s 15ms/step - loss: 0.2505 - accuracy: 0.9045


[0.25048524141311646, 0.9044734239578247]

In [45]:
# embedding_size = 20
# # model_cnn = Sequential()
# create_model
# # model_cnn.add(layers.Embedding(
# #     input_dim=X_pad.shape[0],
# #     input_length=X_pad.shape[1],
# #     output_dim=20,
# #     mask_zero=True))
# model_cnn.add(layers.Conv1D(10, kernel_size=3))
# model_cnn.add(layers.Flatten())
# model_cnn.add(layers.Dense(30, activation="relu"))
# model_cnn.add(layers.Dense(6, activation="softmax"))

In [ ]:
# bi directional lstm
# dropout layers
# model.add(Bidirectional(LSTM(100,return_sequences=True)))